## **1. Regroupement en sous-corpus**
Plutôt que de récupérer un tag qui aurait été assigné manuellement pour chaque lien, on va le créer en déconstruisant les URLs et en créant des sous-dossiers correspondant à différents sous-niveaux dans l'arborescence du site.  

À décider : jusqu'à quel niveau on regroupe (ça peut être un peu du cas par cas selon l'organisation du site)

Créer des sous-dossiers à l'intérieur

In [79]:
path = '../03-corpus/2-data/1-fr/'
acteur = "ciusss_centresud"

from pandas import *

# encoding= 'ISO-8859-1' "utf-8"
with open(path + acteur  + '.csv', encoding='utf-8') as f:
    csv = read_csv(f, sep=',')[["Address", 'Title', 'Type', "text"]]

csv

,Address,Title,Type,text
0,https:////ciusss-centresudmtl.gouv.qc.ca//,CIUSSS du Centre-Sud-de-l&#039;Île-de-Montréal,text//html,Variole simienne Information essentielle conce...
1,https:////ciusss-centresudmtl.gouv.qc.ca//pers...,Personnel et retraités | CIUSSS du Centre-Sud-...,text//html,Équité salariale Pour en connaître davantage s...
2,https:////ciusss-centresudmtl.gouv.qc.ca//prop...,Services en anglais,text//html,À propos Services en anglais et autres langues...
3,https:////ciusss-centresudmtl.gouv.qc.ca//empl...,Emplois et stages | CIUSSS du Centre-Sud-de-l&...,text//html,Emplois et stages Emplois et stages Nous cherc...
4,https:////ciusss-centresudmtl.gouv.qc.ca//nous...,Nous joindre | CIUSSS du Centre-Sud-de-l&#039;...,text//html,Nous joindre Nous joindre Renseignements génér...
...,...,...,...,...
1043,https:////ciusss-centresudmtl.gouv.qc.ca//etab...,Maison de naissance Jeanne-Mance,NaN,"Page introuvable Désolé, il est impossible de ..."
1044,https:////ciusss-centresudmtl.gouv.qc.ca//info...,"<p class=""h2""> Reto...",NaN,Accès refusé Vous n'êtes pas autorisé(e) à acc...
1045,https:////ciusss-centresudmtl.gouv.qc.ca//reto...,"<p class=""h2""> Reto...",NaN,Accès refusé Vous n'êtes pas autorisé(e) à acc...
1046,https:////ciusss-centresudmtl.gouv.qc.ca//nous...,CIUSSS 的网站,NaN,"Page introuvable Désolé, il est impossible de ..."


In [80]:
liste = [lien.replace('//', "/") for lien in csv['Address'].tolist() if not 'actualites' in lien and not 'propos' in lien and not 'joindre' in lien and not 'communiques' in lien and not 'about' in lien and not 'nouvelles' in lien and not 'pdf' in lien and not 'gaspesie' in lien]
liste 

['https://ciusss-centresudmtl.gouv.qc.ca/',
 'https://ciusss-centresudmtl.gouv.qc.ca/personnel-et-retraites',
 'https://ciusss-centresudmtl.gouv.qc.ca/emplois-et-stages',
 'https://ciusss-centresudmtl.gouv.qc.ca/soins-et-services/demande-dacces-au-dossier-de-lusager-et-registre-de-vaccination',
 'https://ciusss-centresudmtl.gouv.qc.ca/partager-son-experience',
 'https://ciusss-centresudmtl.gouv.qc.ca/adresses-et-coordonnees',
 'https://ciusss-centresudmtl.gouv.qc.ca/mission-universitaire',
 'https://ciusss-centresudmtl.gouv.qc.ca/langue-des-signes-quebecoise-lsq-videos',
 'https://ciusss-centresudmtl.gouv.qc.ca/accessibilite',
 'https://ciusss-centresudmtl.gouv.qc.ca/informations-pratiques',
 'https://ciusss-centresudmtl.gouv.qc.ca/soins-et-services/consultation-avec-un-medecin-ou-une-infirmiere/acces-un-medecin-de-famille',
 'https://ciusss-centresudmtl.gouv.qc.ca/soins-et-services/carte-dassurance-maladie',
 'https://ciusss-centresudmtl.gouv.qc.ca/soins-et-services/consultation-avec-

In [81]:
"""
0 = http/s
1 = //
2 = domaine principal (ex. INSPQ)
3 = 1e sous-niveau
4 = 2e sous-niveau
5 = 3e sous-niveau
etc.
"""

liens = []
for i in range (2, 9): # On prend les liens de niveau 2 à 8
    try:
        liens += [lien for lien in liste if len(lien.split('/')) > i]
    except Exception as e:
        True

In [82]:
regroupements = {}

for lien in liens:
    for i in range(3,9):
        try:
            tag = lien.split('/')[i-1]
            if tag in regroupements:
                regroupements[tag].append(lien)
                
            else:
                if len(tag)>1:
                    regroupements[tag] = []

        except Exception as e:
            True

tags = list(regroupements.keys())

In [83]:
tags = [tag for tag in regroupements if len(regroupements[tag]) > 30]

In [84]:
tags

['ciusss-centresudmtl.gouv.qc.ca',
 'personnel-et-retraites',
 'emplois-et-stages',
 'soins-et-services',
 'informations-pratiques',
 'consultation-avec-un-medecin-ou-une-infirmiere',
 'consultation-avec-un-medecin-une-infirmiere-ou-un-travailleur-social',
 'mesures-speciales-pour-les-usagers-et-leurs-proches',
 'milieux-de-vie-des-aines',
 'proches-aidants',
 'signalement-la-dpj',
 'grossesse-et-soins-aux-bebes',
 'actualite',
 'hopital-chinois-de-montreal',
 'postuler-sur-des-affichages-linterne',
 'personnel-de-soutien',
 'personnel-infirmier-et-cardiorespiratoire',
 'professionnels',
 'etablissement',
 'consultation-avec-un-medecin-specialiste',
 'enfants-et-adolescents',
 'aire-ouverte',
 'sante-sexuelle',
 'deficience-physique-sensorielle-langage-auditive-et-motrice',
 'demande-dhebergement',
 'difficultes-familiales-ou-ruptures-conjugales',
 'grossesse',
 'soins-aux-bebes',
 'milieu-hospitalier',
 'sante-mentale-et-dependance',
 'hebergement-permanent',
 'hebergement-chsld',
 'e

In [85]:
print('Ça nous donne {} tags.'.format(len(tags)))
tags

Ça nous donne 36 tags.


['ciusss-centresudmtl.gouv.qc.ca',
 'personnel-et-retraites',
 'emplois-et-stages',
 'soins-et-services',
 'informations-pratiques',
 'consultation-avec-un-medecin-ou-une-infirmiere',
 'consultation-avec-un-medecin-une-infirmiere-ou-un-travailleur-social',
 'mesures-speciales-pour-les-usagers-et-leurs-proches',
 'milieux-de-vie-des-aines',
 'proches-aidants',
 'signalement-la-dpj',
 'grossesse-et-soins-aux-bebes',
 'actualite',
 'hopital-chinois-de-montreal',
 'postuler-sur-des-affichages-linterne',
 'personnel-de-soutien',
 'personnel-infirmier-et-cardiorespiratoire',
 'professionnels',
 'etablissement',
 'consultation-avec-un-medecin-specialiste',
 'enfants-et-adolescents',
 'aire-ouverte',
 'sante-sexuelle',
 'deficience-physique-sensorielle-langage-auditive-et-motrice',
 'demande-dhebergement',
 'difficultes-familiales-ou-ruptures-conjugales',
 'grossesse',
 'soins-aux-bebes',
 'milieu-hospitalier',
 'sante-mentale-et-dependance',
 'hebergement-permanent',
 'hebergement-chsld',
 'e

In [86]:
import os, shutil
from pathlib import Path
from slugify import slugify

# Créer un sous-dossier où on va stocker des nouveaux fichiers CSV correspondant aux sous-corpus
folder_path ='../03-corpus/2-sous-corpus/'

#Path(folder_path).mkdir(parents=True, exist_ok=True)


for tag in tags:
    file_name =  acteur + "_" + slugify(str(tag).strip('\n'))

    # Aller chercher les documents du sous-corpus dans le CSV du corpus principal et les copier dans un nouveau CSV
    try:
        data = csv[csv["Address"].str.match('.*[\-\_\/]'+tag+'.*')]
        data.to_csv(folder_path + file_name + ".csv") #encoding='ISO-8859-1'
    except Exception as e:
        print(tag, str(e))
                    

In [87]:
with open(folder_path + acteur + '_sous-corpus.txt', 'w') as f:
    for file in os.listdir(folder_path):
        f.write(file + '\n')

Après avoir manuellement indexé

In [88]:
from pandas import read_csv
filepath = '../03-corpus/2-sous-corpus/0-semi-automatic_tags_' + acteur + '.csv'
with open(filepath, 'r', encoding = 'utf-8') as f:
    csv = read_csv(f, sep=';')
    
# csv = csv[csv['oui/non'] != 'x']
# csv = csv[csv['oui/non'] != 'X']
csv = csv[csv['Sous-corpus'] != 'x']
csv = csv[csv['Sous-corpus'] != 'X']

csv

,Regroupement dans l'arborescence du site,Sous-corpus
1,ciusss_centresud_aides-techniques-prothese-ort...,Aides techniques / Prothèses et orthèses
3,ciusss_centresud_consultation-avec-un-medecin-...,Soins et services
4,ciusss_centresud_consultation-avec-un-medecin-...,Soins et services
5,ciusss_centresud_consultation-avec-un-medecin-...,Services sociaux
6,ciusss_centresud_deficience-physique-sensoriel...,Déficiences et handicaps
8,ciusss_centresud_etablissement.csv,Établissement
9,ciusss_centresud_grossesse-et-soins-aux-bebes.csv,Grossesse
10,ciusss_centresud_grossesse.csv,Grossesse
12,ciusss_centresud_mesures-speciales-pour-les-us...,Covid-19
13,ciusss_centresud_personnel-et-retraites.csv,Personnel et retraités


In [89]:
# tags = list(set([tag for tag in csv['oui/non'].tolist()]))
tags = list(set([tag for tag in csv['Sous-corpus'].tolist()]))

tags

['Aides techniques / Prothèses et orthèses',
 'Personnel infirmier et cardiorespiratoire',
 'Grossesse',
 'Covid-19',
 'Soins et services',
 'Personnel et retraités',
 'Déficiences et handicaps',
 'Services sociaux',
 'Établissement',
 'Professionnels']

In [90]:
regroupements = {tag : [] for tag in tags}
for tag in tags:
        tuples = csv[csv['Sous-corpus'] == tag]
        liste = [str(t).strip('\n') for t in tuples["Regroupement dans l'arborescence du site"].tolist()]
        regroupements[tag] = liste


In [91]:
regroupements

{'Aides techniques / Prothèses et orthèses': ['ciusss_centresud_aides-techniques-prothese-orthese-etc.csv'],
 'Personnel infirmier et cardiorespiratoire': ['ciusss_centresud_personnel-infirmier-et-cardiorespiratoire.csv'],
 'Grossesse': ['ciusss_centresud_grossesse-et-soins-aux-bebes.csv',
  'ciusss_centresud_grossesse.csv'],
 'Covid-19': ['ciusss_centresud_mesures-speciales-pour-les-usagers-et-leurs-proches.csv'],
 'Soins et services': ['ciusss_centresud_consultation-avec-un-medecin-ou-une-infirmiere.csv',
  'ciusss_centresud_consultation-avec-un-medecin-specialiste.csv',
  'ciusss_centresud_soins-et-services.csv'],
 'Personnel et retraités': ['ciusss_centresud_personnel-et-retraites.csv'],
 'Déficiences et handicaps': ['ciusss_centresud_deficience-physique-sensorielle-langage-auditive-et-motrice.csv'],
 'Services sociaux': ['ciusss_centresud_consultation-avec-un-medecin-une-infirmiere-ou-un-travailleur-social.csv'],
 'Établissement': ['ciusss_centresud_etablissement.csv'],
 'Professi

In [92]:
import pandas
from slugify import slugify

# Créer de nouveaux CSV contenant les données regroupées selon le tag qui leur est associé
path='../03-corpus/2-sous-corpus/'

for tag in regroupements:

    # Aller chercher les éléments correspondant au sous-corpus dans le csv principal et les copier dans un nouveau CSV 
    dfs = []
    path='../03-corpus/2-sous-corpus/'
    for doc in regroupements[tag]:
        with open(path + doc, 'r', encoding='utf-8') as f:
            csv = read_csv(f)
            dfs.append(csv)

    # Concatenate all data into one DataFrame
    try:
        path='../03-corpus/2-data/1-fr/' + acteur +'/'
        file_path = path + acteur + '_'  + slugify(str(tag)) + '.csv'
        big_frame = pandas.concat(dfs, ignore_index=True)
        big_frame['Sous-corpus'] = tag
        big_frame['Corpus'] = acteur
        big_frame = big_frame[["Corpus", "Sous-corpus",	"Address",	"Title",	"Type",	"text"]]
        big_frame.to_csv(file_path)
        print(tag + "\t" + str(len(big_frame)))

    except Exception as e:
        print(tag, str(e))

big_frame

Aides techniques / Prothèses et orthèses	20
Personnel infirmier et cardiorespiratoire	38
Grossesse	64
Covid-19	32
Soins et services	442
Personnel et retraités	30
Déficiences et handicaps	38
Services sociaux	44
Établissement	120
Professionnels	26


,Corpus,Sous-corpus,Address,Title,Type,text
0,ciusss_centresud,Professionnels,https:////ciusss-centresudmtl.gouv.qc.ca//empl...,Professionnels | CIUSSS du Centre-Sud-de-l&#03...,text//html,Emplois et stages Professionnels En choisissan...
1,ciusss_centresud,Professionnels,https:////ciusss-centresudmtl.gouv.qc.ca//empl...,Divers | CIUSSS du Centre-Sud-de-l&#039;Île-de...,text//html,Emplois et stages Divers Préposé(e) en retrait...
2,ciusss_centresud,Professionnels,https:////ciusss-centresudmtl.gouv.qc.ca//empl...,Agents de relations humaines | CIUSSS du Centr...,text//html,Emplois et stages Agents de relations humaines...
3,ciusss_centresud,Professionnels,https:////ciusss-centresudmtl.gouv.qc.ca//empl...,Éducateurs | CIUSSS du Centre-Sud-de-l&#039;Îl...,text//html,Emplois et stages Éducateurs Emplois disponibl...
4,ciusss_centresud,Professionnels,https:////ciusss-centresudmtl.gouv.qc.ca//empl...,Ergothérapeutes | CIUSSS du Centre-Sud-de-l&#0...,text//html,Emplois et stages Ergothérapeutes -
5,ciusss_centresud,Professionnels,https:////ciusss-centresudmtl.gouv.qc.ca//empl...,Orthophonistes | CIUSSS du Centre-Sud-de-l&#03...,text//html,Emplois et stages Orthophonistes Ce que vous d...
6,ciusss_centresud,Professionnels,https:////ciusss-centresudmtl.gouv.qc.ca//empl...,Pharmaciens | CIUSSS du Centre-Sud-de-l&#039;Î...,text//html,Emplois et stages Pharmaciens Emplois disponib...
7,ciusss_centresud,Professionnels,https:////ciusss-centresudmtl.gouv.qc.ca//empl...,Emplois - Professionnels en diététique et nutr...,text//html,Emplois et stages Emplois - Professionnels en ...
8,ciusss_centresud,Professionnels,https:////ciusss-centresudmtl.gouv.qc.ca//empl...,Psychoéducateurs | CIUSSS du Centre-Sud-de-l&#...,text//html,Emplois et stages Psychoéducateurs Clientèle p...
9,ciusss_centresud,Professionnels,https:////ciusss-centresudmtl.gouv.qc.ca//empl...,Physiothérapeutes | CIUSSS du Centre-Sud-de-l&...,text//html,Emplois et stages Physiothérapeutes Plusieurs ...
